# Make dataset

## 데이터 생성에서 사용된 아이디어
1. contrastive learning의 개념을 적용하여 negative sample을 선정하였습니다.
2. 학습할 모델보다 성능이 좋은 다른 모델(UniXcoder)을 이용해서 dataset을 sampling을 하였습니다. 이는 학습모델(GraphCodeBert)이 생각하지 못한 경우를 다른 모델로부터 배울 수 있다는 가정에서 접근하였습니다.
3. 전처리는 간단하게 주석만 제거했습니다.

In [1]:
import jsonlines
import pandas as pd
from tqdm import tqdm
import glob

## code/problem\*/\*.py -> 전처리 -> preprocess-code.csv

In [2]:
file_names = glob.glob('../data/code/**/*.py')
print(len(file_names))

45101


In [3]:
def preprocess(lines):
    l = []
    for line in lines:
        # 주석으로 시작하는 줄 제거
        if line.lstrip().startswith('#'):
            continue
            
        # isBlank
        if not line:
            continue
        
        # 우측 주석 제거
        line = line.rstrip()
        if '#' in line:
            line = line[:line.rindex('#')]
        
        l.append(line)
    return ' '.join(l)

In [4]:
l = []
for file_name in file_names:
    lines = open(file_name).read().splitlines()
    t = {
        'problem_no': int(file_name.split('/')[-2][7:]),
        'code_no': int(file_name.split('/')[-1].split('_')[1][:-3]),
        'code': preprocess(lines)
    }
    l.append(t)

In [5]:
df_code_all = pd.DataFrame.from_dict(l)
df_code_all = df_code_all.sort_values(by=['problem_no', 'code_no'], axis=0)
df_code_all = df_code_all.reset_index(drop=True)
df_code_all = df_code_all.reset_index()
df_code_all.to_csv('./dataset/preprocess-code.csv')
df_code_all

,index,problem_no,code_no,code
0,0,1,1,"M = 9 N = 9 def main(): for i in range(1,M..."
1,1,1,2,"for i in range(1,10): for j in range(1,10)..."
2,2,1,3,"for i in range(1, 10): \tfor ii in range(1, 10..."
3,3,1,4,"for a in range(1,10): for b in range(1,10)..."
4,4,1,5,"for a in range(1,10): for b in range(1,10)..."
...,...,...,...,...
45096,45096,300,146,"def main(): A,B = map(int,input().split()) ..."
45097,45097,300,147,"A, B = list(map(int, input().split())) import ..."
45098,45098,300,148,"A, B = map(int, input().split()) def factoriza..."
45099,45099,300,149,"import math def gcd(a, b): if (b == 0): ..."


## jsonl 생성

In [390]:
l = []
for idx in range(len(df_code_all)):
    d = {
        'func': df_code_all.iloc[idx]['code'],
        'idx': str(df_code_all.iloc[idx]['index'])
    }
    l.append(d)

with jsonlines.open('./dataset/data.jsonl', mode='w') as writer:
    writer.write_all(l)

In [391]:
with jsonlines.open('./dataset/data.jsonl') as f:
    for line in f:
        print(line)
        print(line['func'])
        print(line['idx'])
        print(type(line['idx']))
        break

{'func': 'M = 9 N = 9 def main():     for i in range(1,M+1,1):         for j in range(1,N+1,1):             mult = i * j             print(str(i) + "x" + str(j) + "=" + str(i * j)) main()', 'idx': '0'}
M = 9 N = 9 def main():     for i in range(1,M+1,1):         for j in range(1,N+1,1):             mult = i * j             print(str(i) + "x" + str(j) + "=" + str(i * j)) main()
0
<class 'str'>


## train/valid pair sampling

- 가진 데이터는 300개의 문제에 대해서 소스코드가 150개씩 있습니다.
- train과 valid set을 8:2로 분리합니다.
- 문제 번호(problem_no) 간 데이터 비율을 유지하기 위해서 코드 번호(code_no)를 기준으로 train과 valid를 구분하였습니다.
- 코드 번호(code_no)를 기준으로 120번 이하는 train, 그 외는 valid로 분리하였습니다.

###  negative pair sampling based UniXcoder

- negative sample을 선정합니다.
- contrastive learning 일부 개념을 적용하였습니다.
    - 벡터 공간에서 negative pair는 멀게, positive pair는 가깝게
- [UniXcoder](https://github.com/microsoft/CodeBERT/tree/master/UniXcoder)를 인코더로 이용하여 모든 코드를 동일한 벡터공간에 표현하였습니다.
- 하나의 코드에 대해서 cosine similarity가 높지만 label이 0 코드와 cosine similarty가 0 이하지만 label이 1인 코드를 negative pair로 선택하였습니다.
- 데이터 갯수와 학습시간은 비례하기때문에 negative sample 개수는 하나의 문제당 (전체 문제 수 x 0.0003)개를 선택했습니다. 
- 총 120 x 300 x 11 = 396,000개 중에서 중복제거를 통해 340,071개의 negative pair for training을 선정했습니다.

### positive pair sampling - chaining + random
- positive pair sampling은 chaining으로 일부를 생성하고, negative pair와 수를 맞추기위해 나머지를 random sampling으로 채웠습니다.
- chaining
    - chaining은 제가 명명한 것입니다.
    - 코드를 순서대로 2개씩 pairing하는 것이 chain과 같아서 chaining이라고 명명하였습니다.
    - 하나의 pair를 (문제번호-코드번호, 문제번호-코드번호) 라고 표현한다면 (1-1,1-2), (1-2,1-3) ..이런식으로 pair를 구성하였습니다.
    - chaining을 통해 모든 문제가 최소 1번은 positive data에 추가되도록 하였습니다.
- negative sampling과 수를 맞추기위하여 len(negative_sample) - len(positive_chain_sample) 만큼 random pair를 선택하였습니다.

In [ ]:
# !wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

In [6]:
from unixcoder import UniXcoder
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UniXcoder("microsoft/unixcoder-base")
# model = UniXcoder("microsoft/unixcoder-base-unimodal")
model.to(device)

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [8]:
import itertools
from itertools import combinations
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
import numpy as np

def make_data(df_set, set_number, dataset_type):
    code_embeddings = []
    for text in tqdm(df_set['code']):
        tokens_ids = model.tokenize([text],max_length=512,mode="<encoder-only>")
        source_ids = torch.tensor(tokens_ids).to(device)
        _, code_embedding = model(source_ids)
        code_embeddings.append(code_embedding.cpu().detach().numpy()[0])
    cos_sim = cosine_similarity(code_embeddings)

    negative_samples = []
    for i in tqdm(range(len(cos_sim))):
        problem_nos = list(df_set['problem_no'])
        problem_no = problem_nos[i]
        temp_list = []
        index_asc = np.argsort(cos_sim[i])
        for index in index_asc[::-1]: # 내림차순
            c = cos_sim[i][index]
            if (c > 0 and problem_nos[index] != problem_no):
                temp_list.append([i, index])
            if len(temp_list) > len(cos_sim)*0.0003:
                break
        for index in index_asc: # 오름차순
            c = cos_sim[i][index]
            if c > 0:
                break
            if (c <= 0 and problem_nos[index] == problem_no):
                temp_list.append([i, index])

        negative_samples.extend(temp_list)

    temp_negative_samples = []
    for s in negative_samples:
        s.sort()
        temp_negative_samples.append(s)
    
    temp_negative_samples.sort()
    negative_samples = (list(k for k,_ in itertools.groupby(temp_negative_samples)))
    print('negative samples', len(negative_samples))
        
    ## Positive pair sampling
    
    positive_pairs_chain = []

    def make_positive_chain_pair(df):
        for i in range(len(df['index'])-1):
            positive_pairs_chain.append((df.iloc[i]['index'], df.iloc[i+1]['index']))
        positive_pairs_chain.append((df.iloc[i]['index'], df.iloc[0]['index']))
        
    df_set.groupby('problem_no').apply(make_positive_chain_pair)
    print('positive chain', len(positive_pairs_chain))
        
    positive_pairs_all = []
    positive_pairs_random = []
    
    def make_positive_pair(df):
        comb = list(combinations(df['index'], 2))
        positive_pairs_all.extend(comb)
        random.seed(5)
        max_len = (len(negative_samples)-len(positive_pairs_chain)) // (len(set(df_set['problem_no']))-1)
        positive_pairs_random.extend(random.choices(comb, k=min(max_len, len(comb))))
        
    df_set.groupby('problem_no').apply(make_positive_pair)
    print('positive random samples', len(positive_pairs_random), 'all', len(positive_pairs_all))
        
    positive_samples = positive_pairs_chain + positive_pairs_random
    print('positivie samples', len(positive_samples))
    
    positive_df = pd.DataFrame(positive_samples, columns=['id1', 'id2'])
    positive_df['label'] = 1
    negative_df = pd.DataFrame(negative_samples, columns=['id1', 'id2'])
    negative_df['label'] = 0
    
    df = pd.concat([positive_df, negative_df])
    df = shuffle(df)
    
    print('total samples', len(df))
    
    df.to_csv('./dataset/{}-{}.txt'.format(dataset_type, set_number), sep='\t', header=False, index=False)


In [9]:
# train dataset
df_train = df_code_all[(df_code_all['code_no'] <= 120)]
make_data(df_train, 0, 'train')

100%|████████████████████████████████████| 36000/36000 [03:20<00:00, 179.36it/s]


negative samples 340071
positive chain 36000
positive random samples 304800 all 2142000
positivie samples 340800
total samples 680871


In [27]:
# valid dataset
df_valid = df_code_all[(df_code_all['code_no'] > 120)]
make_data(df_valid, 0, 'valid')

100%|██████████████████████████████████████| 9101/9101 [00:12<00:00, 745.58it/s]


negative samples 23647
positive chain 9101
positive random samples 14400 all 133546
positivie samples 23501
total samples 47148
